<a href="https://colab.research.google.com/github/mjmousavi97/Deep-Learning-Tehran-uni/blob/main/HomeWorks/05%20HW/src/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
from google.colab import files
import zipfile
import cv2
import os
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
uploaded = files.upload()

Saving Images.zip to Images.zip


In [5]:
uploaded = files.upload()

Saving Annotations.zip to Annotations.zip


In [11]:
with zipfile.ZipFile('/content/Images.zip', 'r') as zip_ref:
    zip_ref.extractall('/contents')

In [12]:
with zipfile.ZipFile('/content/Annotations.zip', 'r') as zip_ref:
    zip_ref.extractall('content')

In [21]:
file_name = 'airplane_440'
img = cv2.imread(os.path.join('/content/images/Images/Images/', file_name + '.jpg'))

In [25]:
df = pd.read_csv(os.path.join('/content/Annotations/Airplanes_Annotations/',  file_name + '.csv'))

In [26]:
df

,1
0,115 53 219 156
